In [1]:
import os
import sys

from typing import Tuple, List, Optional
from qiskit import QuantumCircuit

script_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(script_dir)

KAK_SYNTHESIS_CACHE: dict[Tuple[Tuple[float, float], ...], QuantumCircuit] = {}
U3_SYNTHESIS_CACHE: dict[Tuple[float, float, float], Tuple[QuantumCircuit, List[str]]] = {}

GRIDSYNTH = "gridsynth"
SOLOVAY_KITAEV = "solovay_kitaev"
MATCHGATE = "matchgate"

In [2]:
import numpy as np
SEED = 0
np.random.seed(SEED)

def random_rz_rxx_circuit(n_qubits, n_gates):
    """
    Build a random circuit composed of Rz and Rxx gates.
    """
    qc = QuantumCircuit(n_qubits)
    
    for _ in range(n_gates):
        gate_type = np.random.choice(["Rxx", "Rz"])
        if gate_type == "Rz":
            q = np.random.randint(0, n_qubits)
            alpha = np.random.uniform(0, 2*np.pi)
            qc.rz(alpha, q)
        elif gate_type == "Rxx":
            q1 = np.random.choice(range(n_qubits))
            q2 = q1 + 1 if q1 < n_qubits - 1 else 0
            alpha = np.random.uniform(0, 2*np.pi)
            qc.rxx(alpha, q1, q2)

    return qc

# Example
n_qubits = 3
n_gates = 15
circuit = random_rz_rxx_circuit(n_qubits, n_gates)
circuit_name = f"matchgate_{n_qubits}q_{n_gates}g"
print(circuit)

                                  ┌──────────────┐┌──────────────┐»
q_0: ─────────────────────────────┤0             ├┤0             ├»
     ┌──────────────┐┌───────────┐│  Rxx(1.8695) ││  Rxx(1.7132) │»
q_1: ┤0             ├┤ Rz(3.918) ├┤1             ├┤1             ├»
     │  Rxx(5.3047) │├───────────┤└──────────────┘└──────────────┘»
q_2: ┤1             ├┤ Rz(5.103) ├────────────────────────────────»
     └──────────────┘└───────────┘                                »
«     ┌──────────────┐              ┌──────────────┐┌──────────────┐»
«q_0: ┤0             ├──────────────┤0             ├┤0             ├»
«     │  Rxx(2.4679) │┌────────────┐│  Rxx(5.2315) ││  Rxx(6.1488) │»
«q_1: ┤1             ├┤ Rz(2.1199) ├┤1             ├┤1             ├»
«     └──────────────┘└────────────┘└──────────────┘└──────────────┘»
«q_2: ──────────────────────────────────────────────────────────────»
«                                                                   »
«                                 

In [3]:

MODE1 = MATCHGATE
MODE2 = MATCHGATE

synthesis_config = {
    "MODE1": MODE1,  # or "solovay_kitaev"
    "MODE2": MODE2,  # or "solovay_kitaev"
    "EPS": 0.01,
}

model_config = {
    "ONLY_INTERACTIONS": True,
}


In [4]:
from scripts.synthesis.global_synthesis import clifford_t_synthesis_matchgate
from scripts.utils import compute_fidelity

synthesized_circuit, t_list = clifford_t_synthesis_matchgate(circuit, epsilon=1e-2)

#print(synthesized_circuit)
fidelity = compute_fidelity(synthesized_circuit, circuit)
print(f"Fidelity between original and synthesized circuit: {fidelity:.12f}")
print(f"T-count per qubit: {t_list}")

Fidelity between original and synthesized circuit: 0.999715824068
T-count per qubit: [158, 158, 158]


In [5]:
from scripts.mergers import merge_contiguous_2q, apply_sequence_merge
merged_circuit = merge_contiguous_2q(circuit, 0, 1)
fidelity_merged = compute_fidelity(merged_circuit, circuit)
print(f"Fidelity between original and merged circuit: {fidelity_merged:.8f}")
print(merged_circuit)

Fidelity between original and merged circuit: 1.00000000
                     ┌───────────┐                              »
q_0: ────────────────┤0          ├──────────────────────────────»
     ┌──────────────┐│  merged₂q │┌──────────────┐┌────────────┐»
q_1: ┤0             ├┤1          ├┤0             ├┤ Rz(3.3764) ├»
     │  Rxx(5.3047) │├───────────┤│  Rxx(2.8996) │├────────────┤»
q_2: ┤1             ├┤ Rz(5.103) ├┤1             ├┤ Rz(4.5279) ├»
     └──────────────┘└───────────┘└──────────────┘└────────────┘»
«     ┌──────────────┐┌────────────┐
«q_0: ┤1             ├┤ Rz(3.5716) ├
«     │              │└────────────┘
«q_1: ┤  Rxx(2.6054) ├──────────────
«     │              │┌────────────┐
«q_2: ┤0             ├┤ Rz(4.8647) ├
«     └──────────────┘└────────────┘


In [6]:
synthesized_circuit_from_merged, t_list = clifford_t_synthesis_matchgate(merged_circuit, epsilon=1e-2)
print(t_list)


[94, 94, 94]


In [7]:
sequence = [(0, 1), (1, 2)]
merged_circuit_seq = apply_sequence_merge(circuit, sequence, refine=False)
print(merged_circuit_seq)
synthesized_circuit_from_merged, t_list = clifford_t_synthesis_matchgate(merged_circuit_seq, epsilon=1e-2)
print(t_list)


                  ┌───────────┐             ┌──────────────┐┌────────────┐
q_0: ─────────────┤0          ├─────────────┤1             ├┤ Rz(3.5716) ├
     ┌───────────┐│  merged₂q │┌───────────┐│              │└────────────┘
q_1: ┤0          ├┤1          ├┤0          ├┤  Rxx(2.6054) ├──────────────
     │  merged₂q │└───────────┘│  merged₂q ││              │┌────────────┐
q_2: ┤1          ├─────────────┤1          ├┤0             ├┤ Rz(4.8647) ├
     └───────────┘             └───────────┘└──────────────┘└────────────┘
[94, 94, 94]


In [8]:
from scripts.agents.greedy_agent import GreedyMergeOptimizer

target_circuit_and_name = (circuit, circuit_name)

#print number of qubits in circuit

print("Number of qubits in circuit:", circuit.num_qubits)

optimizer = GreedyMergeOptimizer(
    synthesis_config=synthesis_config,
    target_circuit_and_name=target_circuit_and_name,
    model_config=model_config,
    KAK_SYNTHESIS_CACHE=KAK_SYNTHESIS_CACHE,
    U3_SYNTHESIS_CACHE=U3_SYNTHESIS_CACHE
)
greedy_sequence, greedy_tcount = optimizer.optimize(verbose=True)
optimizer.report()
print("Greedy sequence:", greedy_sequence)
n_cached_gates_after = len(U3_SYNTHESIS_CACHE)
print("Number of cached U3 gates after optimization:", n_cached_gates_after)


Number of qubits in circuit: 3
Target circuit name: matchgate_3q_15g
Number of qubits in target circuit: 3
Preprocessing done. Circuit cleaned.
First synthesis completed in 0.20 seconds.
========== Initial Circuit Statistics ==========
Number of qubits:       3
Number of gates:        15
Initial T-count:        474
Time taken:             0.20 seconds
Possible merges:        3
Merge strategy:         All qubit pairs considered


Initial T-count: 474


Optimizing:  33%|███▎      | 2/6 [00:00<00:00,  6.22eval/s]

✅ Merge (0, 1) IMPROVES T-count by 192 (new: 282, old: 474)
❌ Merge (0, 2) worsens T-count by 42 (new: 516, old: 474)


Optimizing:  67%|██████▋   | 4/6 [00:00<00:00,  5.48eval/s]

❌ Merge (1, 2) worsens T-count by 48 (new: 522, old: 474)
⭐ Added merge (0, 1), new best T-count: 282
❌ Merge (0, 2) worsens T-count by 60 (new: 342, old: 282)


Optimizing:  83%|████████▎ | 5/6 [00:00<00:00,  5.57eval/s]


❌ Merge (1, 2) worsens T-count by 54 (new: 336, old: 282)
No improving merge found. Terminating.
========== Optimization Report ==========
Applied merges: [(0, 1)]
Final T-count: 282
Final Fidelity: 0.999626
Time taken: 0.02 minutes
========== Optimization Report ==========
Applied merges: [(0, 1)]
Final T-count: 282
Final Fidelity: 0.999626
Time taken: 0.03 minutes
Greedy sequence: [(0, 1)]
Number of cached U3 gates after optimization: 0
